# Data Cleaning

This notebook handles the data cleaning process for the datasets, documenting the process at the same time.

In [1]:
# Libraries

import pandas as pd
from pathlib import Path
from georesolver import PlaceResolver


In [2]:
# Custom modules
from utils.ColumnManager import ColumnManager
from actions.normalizers.DatesNormalizer import DateNormalizer
from actions.normalizers.NamesNormalizer import NamesNormalizer
from actions.generators.AgeInferrer import AgeInferrer

## Raw Datasets

The raw datasets are stored in the `data/raw` directory. The datasets include:

- `bautismos.csv`: Baptism records
- `matrimonios.csv`: Marriage records
- `entierros.csv`: Burial records

In [3]:
BAUTISMOS_RAW = pd.read_csv("../data/raw/bautismos.csv")
MATRIMONIOS_RAW = pd.read_csv("../data/raw/matrimonios.csv")
ENTIERROS_RAW = pd.read_csv("../data/raw/entierros.csv")

BAUTISMOS_RAW.head()

,Secuencia,Unidad Documental Compuesta (a la que pertenece),Identificador (es recomendable seguir una secuencia numeral como la mostrada en los ejemplos),Título (incluir un título breve para cada documento),Folio inicial del documento (convertir como se muestra abajo),Folio final del documento (convertir como se muestra abajo),Imagen inicial (estos valores serán añadidos cuando comienze el proceso de revisión de imágenes),Imagen final (estos valores serán añadidos cuando comienze el proceso de revisión de imágenes),Tipo de evento,Fecha aaaa-mm-dd,...,Condición de la madrina,Lugar de bautizo,Notas adicionales del documento,Descriptor Geográfico 1,Descriptor Geográfico 2,Descriptor Geográfico 3,Descriptor Geográfico 4,5,Características físicas (Estado de conservación de los materiales físicos),Historia de revisión (de los materiales digitalizados)
0,1.0,APAucará LB L001,B001,Bautizo. Domingo. Tributarios,3r,3r,IMG_7000a,IMG_7000a,Bautizo,1790-10-04,...,NaN,"Pampamarca, iglesia",NaN,Aucara,Pampamarca,NaN,NaN,NaN,Regular,Registrado por Edwin Gonzales en 2023
1,2.0,APAucará LB L001,B002,Bautizo. Dominga. Tributarios,3r,3r,IMG_7000a,IMG_7000a,Bautizo,1790-10-06,...,NaN,"Pampamarca, iglesia",NaN,Aucara,Pampamarca,NaN,NaN,NaN,Regular,Registrado por Edwin Gonzales en 2023
2,3.0,APAucará LB L001,B003,Bautizo. Bartola. Tributarios,3r,3r,IMG_7000a,IMG_7000a,Bautizo,1790-10-07,...,NaN,"Pampamarca, iglesia",NaN,Aucara,Pampamarca,NaN,NaN,NaN,Regular,Registrado por Edwin Gonzales en 2023
3,4.0,APAucará LB L001,B004,Bautizo. Francisca,3v,3v,IMG_7000b,IMG_7000b,Bautizo,1790-10-20,...,NaN,"Aucara, iglesia",Abreviatura poco visible en el margen,Aucara,NaN,NaN,NaN,NaN,Regular,Registrado por Edwin Gonzales en 2023
4,5.0,APAucará LB L001,B005,Bautizo. Pedro,3v,3v,IMG_7000b,IMG_7000b,Bautizo,1790-10-20,...,NaN,"Aucara, iglesia",Margen roto y manchado de tinta,Aucara,NaN,NaN,NaN,NaN,Regular,Registrado por Edwin Gonzales en 2023


## Column Harmonization

Rename the columns in the datasets to ensure consistency across different data sources.

Values are mapped using the `json` files located in the `data/mappings` directory.

In [4]:
bautismoMapping = Path("../data/mappings/bautismosMapping.json")
matrimonioMapping = Path("../data/mappings/matrimoniosMapping.json")
entierroMapping = Path("../data/mappings/entierrosMapping.json")

column_manager = ColumnManager()

BAUTISMOS_HARMONIZED = column_manager.harmonize_columns(BAUTISMOS_RAW, bautismoMapping)
MATRIMONIOS_HARMONIZED = column_manager.harmonize_columns(MATRIMONIOS_RAW, matrimonioMapping)
ENTIERROS_HARMONIZED = column_manager.harmonize_columns(ENTIERROS_RAW, entierroMapping)

BAUTISMOS_HARMONIZED.head()

,id,file,identifier,title,start_folio,end_folio,start_image,end_image,event_type,date,...,godmother_social_condition,baptism_place,additional_notes,geographic_descriptor_1,geographic_descriptor_2,geographic_descriptor_3,geographic_descriptor_4,other,record_physical_characteristics,revision_history
0,1.0,APAucará LB L001,B001,Bautizo. Domingo. Tributarios,3r,3r,IMG_7000a,IMG_7000a,Bautizo,1790-10-04,...,NaN,"Pampamarca, iglesia",NaN,Aucara,Pampamarca,NaN,NaN,NaN,Regular,Registrado por Edwin Gonzales en 2023
1,2.0,APAucará LB L001,B002,Bautizo. Dominga. Tributarios,3r,3r,IMG_7000a,IMG_7000a,Bautizo,1790-10-06,...,NaN,"Pampamarca, iglesia",NaN,Aucara,Pampamarca,NaN,NaN,NaN,Regular,Registrado por Edwin Gonzales en 2023
2,3.0,APAucará LB L001,B003,Bautizo. Bartola. Tributarios,3r,3r,IMG_7000a,IMG_7000a,Bautizo,1790-10-07,...,NaN,"Pampamarca, iglesia",NaN,Aucara,Pampamarca,NaN,NaN,NaN,Regular,Registrado por Edwin Gonzales en 2023
3,4.0,APAucará LB L001,B004,Bautizo. Francisca,3v,3v,IMG_7000b,IMG_7000b,Bautizo,1790-10-20,...,NaN,"Aucara, iglesia",Abreviatura poco visible en el margen,Aucara,NaN,NaN,NaN,NaN,Regular,Registrado por Edwin Gonzales en 2023
4,5.0,APAucará LB L001,B005,Bautizo. Pedro,3v,3v,IMG_7000b,IMG_7000b,Bautizo,1790-10-20,...,NaN,"Aucara, iglesia",Margen roto y manchado de tinta,Aucara,NaN,NaN,NaN,NaN,Regular,Registrado por Edwin Gonzales en 2023


### Reduce DataFrames to their relevant columns

In [5]:
# Bautismos useful columns

BAUTISMOS_HARMONIZED = BAUTISMOS_HARMONIZED[
    [
        'date', 'baptized_name', 'hometown', 'birth_date', 'baptized_legitimacy_status',
        'father_name', 'father_lastname', 'father_social_condition', 'mother_name',
        'mother_lastname', 'mother_social_condition', 'parents_social_condition',
        'godfather_name', 'godfather_lastname', 'godfather_social_condition',
        'godmother_name', 'godmother_lastname', 'godmother_social_condition',
        'baptism_place', 'additional_notes', 'geographic_descriptor_1',
        'geographic_descriptor_2', 'geographic_descriptor_3',
        'geographic_descriptor_4', 'other'
    ]
]

# remove empty columns
BAUTISMOS_HARMONIZED.dropna(axis=1, how='all', inplace=True)

BAUTISMOS_HARMONIZED.columns

Index(['date', 'baptized_name', 'hometown', 'birth_date',
       'baptized_legitimacy_status', 'father_name', 'father_lastname',
       'father_social_condition', 'mother_name', 'mother_lastname',
       'mother_social_condition', 'parents_social_condition', 'godfather_name',
       'godfather_lastname', 'godfather_social_condition', 'godmother_name',
       'godmother_lastname', 'godmother_social_condition', 'baptism_place',
       'additional_notes', 'geographic_descriptor_1',
       'geographic_descriptor_2', 'geographic_descriptor_3',
       'geographic_descriptor_4'],
      dtype='object')

In [6]:
MATRIMONIOS_HARMONIZED = MATRIMONIOS_HARMONIZED[
    ['date',
       'groom_name', 'groom_lastname', 'groom_social_condition',
       'groom_marital_status', 'groom_age', 'groom_hometown',
       'groom_resident_in', 'groom_legitimacy_status', 'groom_father_name',
       'groom_father_lastname', 'groom_father_social_condition',
       'groom_mother_name', 'groom_mother_lastname',
       'groom_mother_social_condition', 'bride_name', 'bride_lastname',
       'bride_social_condition', 'bride_marital_status', 'bride_age',
       'bride_hometown', 'bride_resident_in', 'bride_legitimacy_status',
       'bride_father_name', 'bride_father_lastname',
       'bride_father_social_condition', 'bride_mother_name',
       'bride_mother_lastname', 'bride_mother_social_condition',
       'godparent_1_name', 'godparent_1_lastname',
       'godparent_1_social_condition', 'godparent_2_name',
       'godparent_2_lastname', 'godparent_2_social_condition',
       'godparent_3_name', 'godparent_3_lastname',
       'godparent_3_social_condition', 'witness_1_name', 'witness_1_lastname',
       'witness_2_name', 'witness_2_lastname', 'witness_3_name',
       'witness_3_lastname', 'witness_4_name', 'witness_4_lastname',
       'marriage_place', 'geographic_descriptor_1', 'geographic_descriptor_2',
       'geographic_descriptor_3', 'geographic_descriptor_4',
       'geographic_descriptor_5', 'geographic_descriptor_6']
]

# remove empty columns
MATRIMONIOS_HARMONIZED.dropna(axis=1, how='all', inplace=True)

MATRIMONIOS_HARMONIZED.columns

Index(['date', 'groom_name', 'groom_lastname', 'groom_social_condition',
       'groom_marital_status', 'groom_age', 'groom_hometown',
       'groom_resident_in', 'groom_legitimacy_status', 'groom_father_name',
       'groom_father_lastname', 'groom_father_social_condition',
       'groom_mother_name', 'groom_mother_lastname',
       'groom_mother_social_condition', 'bride_name', 'bride_lastname',
       'bride_social_condition', 'bride_marital_status', 'bride_age',
       'bride_hometown', 'bride_resident_in', 'bride_legitimacy_status',
       'bride_father_name', 'bride_father_lastname',
       'bride_father_social_condition', 'bride_mother_name',
       'bride_mother_lastname', 'bride_mother_social_condition',
       'godparent_1_name', 'godparent_1_lastname',
       'godparent_1_social_condition', 'godparent_2_name',
       'godparent_2_lastname', 'godparent_2_social_condition',
       'godparent_3_name', 'godparent_3_lastname', 'witness_1_name',
       'witness_1_lastname', 'witne

In [7]:
ENTIERROS_HARMONIZED = ENTIERROS_HARMONIZED[
    ['date',
       'doctrine', 'place', 'deceased_name',
       'deceased_lastname', 'age', 'hometown', 'condition', 'marital_status',
       'deceased_legitimacy_status', 'father_name', 'father_lastname',
       'mother_name', 'mother_lastname', 'husband_name', 'wife_name',
       'cause_of_death', 'burial_place', 'geographic_descriptor_1',
       'geographic_descriptor_2', 'geographic_descriptor_3',
       'geographic_descriptor_4',]
]

# remove empty columns
ENTIERROS_HARMONIZED.dropna(axis=1, how='all', inplace=True)

ENTIERROS_HARMONIZED.columns

Index(['date', 'doctrine', 'place', 'deceased_name', 'deceased_lastname',
       'age', 'hometown', 'condition', 'marital_status',
       'deceased_legitimacy_status', 'father_name', 'father_lastname',
       'mother_name', 'mother_lastname', 'husband_name', 'wife_name',
       'cause_of_death', 'burial_place', 'geographic_descriptor_1',
       'geographic_descriptor_2', 'geographic_descriptor_3',
       'geographic_descriptor_4'],
      dtype='object')

## Dates Normalization

Ensure dates are in a consistent format across all datasets. The dates should be in the format `YYYY-MM-DD`.

In [8]:
BAUTISMOS_HARMONIZED['date'] = DateNormalizer(BAUTISMOS_HARMONIZED['date']).normalize()
BAUTISMOS_HARMONIZED['date']

0       1790-10-04
1       1790-10-06
2       1790-10-07
3       1790-10-20
4       1790-10-20
           ...    
6336    1888-12-10
6337    1888-12-11
6338    1888-12-12
6339    1888-12-15
6340    1888-12-16
Name: date, Length: 6341, dtype: object

In [9]:
MATRIMONIOS_HARMONIZED['date'] = DateNormalizer(MATRIMONIOS_HARMONIZED['date']).normalize()
MATRIMONIOS_HARMONIZED['date']

0       1816-12-06
1       1816-12-12
2       1817-03-05
3       1817-03-10
4       1817-03-12
           ...    
1714    1907-10-27
1715    1908-01-13
1716    1908-01-15
1717    1908-02-15
1718    1908-03-17
Name: date, Length: 1719, dtype: object

In [10]:
ENTIERROS_HARMONIZED['date'] = DateNormalizer(ENTIERROS_HARMONIZED['date']).normalize()
ENTIERROS_HARMONIZED['date']

0       1846-10-06
1       1846-10-07
2       1846-11-02
3       1846-12-08
4       1847-02-23
           ...    
2193    1920-10-12
2194    1920-10-19
2195    1920-10-19
2196    1920-10-20
2197    1920-10-21
Name: date, Length: 2198, dtype: object

## Age Inferring

Infer the age of individuals based on their birth dates and the date of the event (baptism, marriage, burial).

In [11]:
BAUTISMOS_HARMONIZED['birth_date'] = AgeInferrer(BAUTISMOS_HARMONIZED['date']).infer_all(BAUTISMOS_HARMONIZED['birth_date'])
BAUTISMOS_HARMONIZED[['date', 'birth_date']]

,date,birth_date
0,1790-10-04,1790-08-04
1,1790-10-06,1790-08-04
2,1790-10-07,1790-08-04
3,1790-10-20,1790-10-15
4,1790-10-20,1790-10-19
...,...,...
6336,1888-12-10,1888-12-09
6337,1888-12-11,1888-12-07
6338,1888-12-12,1888-12-06
6339,1888-12-15,1888-11-30


In [12]:
# Create boolean mask for invalid records
invalid_mask = pd.to_datetime(BAUTISMOS_HARMONIZED['date'], errors='coerce') < pd.to_datetime(BAUTISMOS_HARMONIZED['birth_date'], errors='coerce')
if invalid_mask.any():
    print("Found invalid records:")
    print(BAUTISMOS_HARMONIZED[invalid_mask][['date', 'birth_date']])

Found invalid records:
            date  birth_date
135   1792-03-29  1792-04-08
290   1794-01-01  1794-01-27
671   1797-07-15  1797-07-24
2814  1900-04-01  1900-04-09


In [13]:
BAUTISMOS_HARMONIZED[['date', 'birth_date']].iloc[930:955]

,date,birth_date
930,1802-09-19,1802-05-22
931,1802-09-22,NaN
932,1802-10-08,1802-10-07
933,1802-10-14,1802-10-13
934,1802-10-23,NaN
935,1802-11-08,1802-10-24
936,1802-11-28,1802-09-29
937,1802-11-28,1802-11-23
938,1802-12-09,1802-11-24
939,1802-12-09,1802-11-09


## Names Normalization

Standardize names across datasets to ensure consistency. This includes normalizing first names, last names, and any other relevant name fields.

In [14]:
def normalize_names_columns(series):
    namesManager = NamesNormalizer()
    return namesManager.clean_series(series)

In [15]:
names_columns = [
    'baptized_name', 
    'father_name', 'father_lastname',
    'mother_name', 'mother_lastname',
    'godfather_name', 'godfather_lastname', 
    'godmother_name', 'godmother_lastname',
]

for col in names_columns:
    if col in BAUTISMOS_HARMONIZED.columns:
        BAUTISMOS_HARMONIZED[col] = normalize_names_columns(BAUTISMOS_HARMONIZED[col])

BAUTISMOS_HARMONIZED[names_columns]

,baptized_name,father_name,father_lastname,mother_name,mother_lastname,godfather_name,godfather_lastname,godmother_name,godmother_lastname
0,domingo,lucas,ayquipa,sevastiana,quispe,vicente,guamani,None,None
1,dominga,juan,lulia,jospha,gomes,ignacio,varientos,None,None
2,bartola,jacinto,quispe,juliana,chinchay,None,None,rotonda,pocco
3,francisca,juan,cuebas,clemenzia,manco,None,None,ysabel,guillen
4,pedro,santos,manxo,baleriana,arango,None,None,josefa,santiago
...,...,...,...,...,...,...,...,...,...
6336,leocadio,miguel,pacheco,rosa,huarcaya,josé julián,bendezú,None,None
6337,mariano concepcion,facundo,vega,silvestra,urbano,fernando,mancco,None,None
6338,ambrosio,ysidro,ccasane,rita,palomino,juan,tito,None,None
6339,francisco,mariano,lopez,leocadia,medina,feliciano,dias,None,None


In [16]:
matrimonios_names_columns = [
    'groom_name', 'groom_lastname', 
       'groom_father_name', 'groom_father_lastname', 
       'groom_mother_name', 'groom_mother_lastname',
       'bride_name', 'bride_lastname',
       'bride_father_name', 'bride_father_lastname',
       'bride_mother_name', 'bride_mother_lastname', 
       'godparent_1_name', 'godparent_1_lastname',
       'godparent_2_name', 'godparent_2_lastname', 
       'godparent_3_name', 'godparent_3_lastname', 
       'witness_1_name', 'witness_1_lastname', 
       'witness_2_name', 'witness_2_lastname',
       'witness_3_name', 'witness_3_lastname', 
       'witness_4_name', 'witness_4_lastname'
]

for col in matrimonios_names_columns:
    if col in MATRIMONIOS_HARMONIZED.columns:
        MATRIMONIOS_HARMONIZED[col] = normalize_names_columns(MATRIMONIOS_HARMONIZED[col])

MATRIMONIOS_HARMONIZED[matrimonios_names_columns]

,groom_name,groom_lastname,groom_father_name,groom_father_lastname,groom_mother_name,groom_mother_lastname,bride_name,bride_lastname,bride_father_name,bride_father_lastname,...,godparent_3_name,godparent_3_lastname,witness_1_name,witness_1_lastname,witness_2_name,witness_2_lastname,witness_3_name,witness_3_lastname,witness_4_name,witness_4_lastname
0,josé manl manuel,de la roca,acencio,roca,leonor,guerrero,juana,rodrigues,pedro,rodrigues,...,None,None,agustin,castro,mariano,castro,juan,baldes,None,None
1,esteban,castillo,matheo,castillo,ma maria,torres,ambrocia,tasqui,pedro,tasqui,...,None,None,pedro,manco,carlos,canto,pedro,guamani,None,None
2,alexandro,ramires,leonor,romani,franca francisca,paucar,sipriana,coillo,cristobal,coillo,...,None,None,marcelo,llamuca,julian,urbano,antonio,urbano,None,None
3,jose,cuchu,acencio,cuchu,baleriana,antay,cacimira,flores,None,None,...,None,None,pablo,roque,antonio,urbano,cristobal,coillo,None,None
4,domingo,tito,None,None,marcela,guauya,petrona,guallpatuiru,agustin,guallpatuiru,...,None,None,marcelo,llamuca,antonio,guamani,mariano,guallpatuiru,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,patrocinio,chinchay,miguel,chinchay,andrea,polanco,lorenza,quispe,gervacio,quispe,...,None,None,juan de dios,barrientos,manuel,espinosa,crisostomo,pumarino,None,None
1715,gerónimo,cucho,ambrocio,cucho,gertrudis,serrano,teresa,jimenes,aniseto,jimenes,...,None,None,victor,saravia,mateo,aiquipa,felix,cucho,None,None
1716,josé,coro,felix,coro,natividad,cucho,emilia,huamani,patricio,huamani,...,None,None,pablo,de la cruz,victor,saravia,marcelo,ramos,None,None
1717,pedro,gutierres,ruperto,gutierrez,micaila,oscco,juliana,huarcaya,hilario,huarcaya,...,None,None,rafael,delgado,josé,vivanco,agustin,vicente,None,None
